<a href="https://colab.research.google.com/github/sergioopereira/AI/blob/main/01_DirecaoVerificada_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import numpy as np
import pandas as pd
from datetime import datetime,date,time
from dateutil.relativedelta import relativedelta


### **A) Coleta dos Dados de Direção do Vento**

In [52]:
source_file = 'raw/BAUET2_CAETITÉ_2_DIR_Verif_EPE.txt'

In [53]:
df_direcao = pd.read_csv(source_file, delimiter = ';' , header=None)

### **B) Sobre os Dados no Patio 1**

#### <span style="color:#DC143C">**B1.Formato dos Dados**

In [54]:
df_direcao.shape # formato da matriz

(1622, 49)

#### <span style="color:#DC143C">**B2.Visao do Dataframe**

In [55]:
df_direcao.head(2)  # visão de 5 linhas

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
0,20170101,999.0,999.0,116.567,121.833,120.600,121.367,122.767,116.667,116.367,117.067,116.367,117.933,117.600,118.133,118.133,115.233,115.733,115.300,113.200,111.1,109.133,120.967,113.6,109.033,111.533,102.433,116.667,112.333,102.967,103.533,104.367,98.000,109.867,114.100,118.3,113.167,105.100,108.933,114.833,115.567,113.400,110.633,102.4,97.300,95.167,120.733,106.967,104.500
1,20170102,117.7,116.3,105.267,108.133,118.467,113.400,110.600,117.200,120.267,113.367,111.633,117.567,108.533,110.200,117.767,118.333,114.400,107.767,109.533,110.7,106.300,103.733,109.5,120.933,116.800,128.567,124.333,137.100,137.467,123.067,124.167,138.067,124.167,128.533,136.9,109.000,121.667,88.500,104.867,111.867,107.067,105.367,96.0,80.367,65.000,115.300,107.933,97.667


### **C) Extração os Dados de 2017**

#### <span style="color:#DC143C"> **C1. Converter a primeira coluna em Data**

In [56]:
df_direcao[0]= pd.to_datetime(df_direcao[0], format='%Y%m%d')

#### <span style="color:#DC143C"> **C2.Remove registros fora do Intervalo**

In [57]:
df_direcao[df_direcao[0] > '2017-12-31'].index

Int64Index([ 365,  366,  367,  368,  369,  370,  371,  372,  373,  374,
            ...
            1612, 1613, 1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621],
           dtype='int64', length=1257)

In [58]:
df_direcao = df_direcao.drop(df_direcao[df_direcao[0] > '2017-12-31'].index)
df_direcao = df_direcao.drop(df_direcao[df_direcao[0] < '2017-01-01'].index)

In [59]:
df_direcao['longitude'] = -42.75
df_direcao['latitude']  = -14.25 

In [60]:
cols = list(df_direcao)
cols = [cols[-1]] + cols[:-1]
df_direcao = df_direcao[cols]

cols = list(df_direcao)
cols = [cols[-1]] + cols[:-1]
df_direcao = df_direcao[cols]

In [61]:
df_direcao = df_direcao.reset_index()

### **D) Formatação dos Dados**

#### <span style="color:#DC143C"> **D1. Migrar os dados para o Formato**

In [62]:
frente = ['Longitude', 'Latitude', 'Data', 'Valor30_1', 'Valor30_2']  # Cabeçalho
df_result_direcao = pd.DataFrame(columns=frente)  # Criação de um dataframe vazio
    
Linha1    =  df_direcao.shape[0] 
Colunas1  =  df_direcao.shape[1] 

for lin2 in range(Linha1):  
    refhora = 0
    
    for col in range (Colunas1):        
        if col > 0:            
        
            if col % 2 != 0: # Se o número (col) for impar                   
               
                if col < 49:
                    
                    Data = df_direcao.loc[lin2,0]
                                  
                    hora = time(hour=refhora, minute=0, second=0)            
                    dataCompleta = datetime.combine(Data, hora)
                    #print(dataCompleta)
                    lin_alvo = [df_direcao.loc[lin2,'longitude'], df_direcao.loc[lin2,'latitude'], dataCompleta , df_direcao.loc[lin2,col], df_direcao.loc[lin2,col+1]] 
                    # print(lin_alvo)
                    df_result_direcao.loc[len(df_result_direcao)] = lin_alvo  # adding a row
                         
                refhora = refhora + 1          
                


#### <span style="color:#DC143C"> **D2. Informações sobre os Tipos de Dados**

In [63]:
df_result_direcao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8760 entries, 0 to 8759
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Longitude  8760 non-null   float64       
 1   Latitude   8760 non-null   float64       
 2   Data       8760 non-null   datetime64[ns]
 3   Valor30_1  8760 non-null   float64       
 4   Valor30_2  8760 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 410.6 KB


In [64]:
df_result_direcao.shape

(8760, 5)

save_file = 'd:\df_result_direcao.csv'
df_result_direcao.to_csv(save_file, index = True)

#### <span style="color:#DC143C"> **D3. Remover ruido**

In [65]:
df_result_direcao

,Longitude,Latitude,Data,Valor30_1,Valor30_2
0,-42.75,-14.25,2017-01-01 00:00:00,999.000,999.000
1,-42.75,-14.25,2017-01-01 01:00:00,116.567,121.833
2,-42.75,-14.25,2017-01-01 02:00:00,120.600,121.367
3,-42.75,-14.25,2017-01-01 03:00:00,122.767,116.667
4,-42.75,-14.25,2017-01-01 04:00:00,116.367,117.067
...,...,...,...,...,...
8755,-42.75,-14.25,2017-12-31 19:00:00,85.667,89.667
8756,-42.75,-14.25,2017-12-31 20:00:00,83.667,76.000
8757,-42.75,-14.25,2017-12-31 21:00:00,80.000,85.000
8758,-42.75,-14.25,2017-12-31 22:00:00,85.667,84.667


In [66]:
# Aplicando uma média
# df_result_direcao.loc[df_result_direcao['Valor30_1']== 999.00,'Valor30_1'] = 122.767
# df_result_direcao.loc[df_result_direcao['Valor30_2']== 999.000,'Valor30_2'] = 116.667


In [67]:
# Removendo as linhas sem valores
df_result_direcao = df_result_direcao.drop(df_result_direcao[df_result_direcao['Valor30_1'] == 999.00].index)
df_result_direcao = df_result_direcao.drop(df_result_direcao[df_result_direcao['Valor30_2'] == 999.00].index)

In [68]:
df_result_direcao

,Longitude,Latitude,Data,Valor30_1,Valor30_2
1,-42.75,-14.25,2017-01-01 01:00:00,116.567,121.833
2,-42.75,-14.25,2017-01-01 02:00:00,120.600,121.367
3,-42.75,-14.25,2017-01-01 03:00:00,122.767,116.667
4,-42.75,-14.25,2017-01-01 04:00:00,116.367,117.067
5,-42.75,-14.25,2017-01-01 05:00:00,116.367,117.933
...,...,...,...,...,...
8754,-42.75,-14.25,2017-12-31 18:00:00,81.000,79.000
8755,-42.75,-14.25,2017-12-31 19:00:00,85.667,89.667
8756,-42.75,-14.25,2017-12-31 20:00:00,83.667,76.000
8757,-42.75,-14.25,2017-12-31 21:00:00,80.000,85.000


#### <span style="color:#DC143C"> **D3. Agregar Magnitude para 1H**

In [69]:
df_result_direcao['Direcao_Verif'] = (df_result_direcao['Valor30_1'] + df_result_direcao['Valor30_2']) / 2

#### <span style="color:#DC143C"> **D5. Formata Colunas**

In [70]:
# Formatar o resultado com 2 casas decimais
df_result_direcao['Direcao_Verif'] = df_result_direcao['Direcao_Verif'].round(decimals=2)

In [71]:
df_result_direcao = df_result_direcao.drop('Valor30_1', 1)
df_result_direcao = df_result_direcao.drop('Valor30_2', 1)

In [72]:
df_result_direcao.head(10)

,Longitude,Latitude,Data,Direcao_Verif
1,-42.75,-14.25,2017-01-01 01:00:00,119.20
2,-42.75,-14.25,2017-01-01 02:00:00,120.98
3,-42.75,-14.25,2017-01-01 03:00:00,119.72
4,-42.75,-14.25,2017-01-01 04:00:00,116.72
5,-42.75,-14.25,2017-01-01 05:00:00,117.15
6,-42.75,-14.25,2017-01-01 06:00:00,117.87
7,-42.75,-14.25,2017-01-01 07:00:00,116.68
8,-42.75,-14.25,2017-01-01 08:00:00,115.52
9,-42.75,-14.25,2017-01-01 09:00:00,112.15
10,-42.75,-14.25,2017-01-01 10:00:00,115.05


### **E. Exporta Resultados**

In [73]:
output_file = 'processed/teste_03_patio_direcao.csv'
df_result_direcao.to_csv(output_file, sep=';', encoding='utf-8')